In [1]:
!nvidia-smi

Fri Nov 26 16:09:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 67%   48C    P8    49W / 370W |     45MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
|  0%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


In [3]:
pos = 27297*2
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.73
Weight for class 1: 1.59


In [4]:
IMG_SIZE = 256

# Create a generator

data_augmentation_1 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.01, 0.01), fill_mode='reflect')
])

data_augmentation_2 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.005, 0.005), fill_mode='reflect')
])

## Augmentation

In [5]:
BATCH_SIZE = 32
checkpoint_filepath = 'checkpoints/AUC/checkpoint_aug'
archi = 'Dnet121'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

In [6]:
record_file_train = 'copd_train_new.tfrecords'
train_majority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases
                  
train_minority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

train_majority = train_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

train_dataset = train_majority.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_dataset.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_dataset.shuffle(2*pos+neg).batch(BATCH_SIZE)

In [7]:
record_file_val = 'copd_val_new.tfrecords'
val_majority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases

val_minority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

val_majority = val_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

val_dataset = val_majority.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = val_dataset.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = val_dataset.shuffle(42908).batch(BATCH_SIZE)

In [ ]:
model = load_model_from_pretrain(archi)

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback)

gc.collect()

Epoch 1/20
5429/5429 [==============================] - 682s 108ms/step - loss: 0.5421 - auc: 0.7383 - val_loss: 0.8572 - val_auc: 0.7177
Epoch 2/20
5429/5429 [==============================] - 665s 106ms/step - loss: 0.5062 - auc: 0.7833 - val_loss: 0.5407 - val_auc: 0.7281
Epoch 3/20
5429/5429 [==============================] - 660s 105ms/step - loss: 0.4758 - auc: 0.8148 - val_loss: 0.5723 - val_auc: 0.7296
Epoch 4/20
3854/5429 [====================>.........] - ETA: 2:23 - loss: 0.4372 - auc: 0.8485

In [ ]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_aug'
archi = 'Dnet121'

model = define_model(archi)

model.load_weights(checkpoint_filepath)

In [13]:
record_file = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.batch(32)) # filter out negative cases

y_preds = model.predict(test_dataset)

In [14]:
y_test = get_data_label('test')

test_CI(y_preds, y_test)

print(average_precision_score(y_test, y_preds, average=None))
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

gc.collect()

Sample auc mean: 0.72
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.71 - 0.72]
0.3666053301222415
thresh: 0.26949644
              precision    recall  f1-score   support

           0       0.89      0.63      0.74     36808
           1       0.31      0.69      0.43      9026

    accuracy                           0.64     45834
   macro avg       0.60      0.66      0.58     45834
weighted avg       0.78      0.64      0.68     45834

thresh: 0.26949644
              precision    recall  f1-score   support

           0       0.89      0.63      0.74     36808
           1       0.31      0.69      0.43      9026

    accuracy                           0.64     45834
   macro avg       0.60      0.66      0.58     45834
weighted avg       0.78      0.64      0.68     45834

thresh: 0.5
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     36808
           1       0.40      0.41      0.40      9026

    acc

0